### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)
    
from api.blp import BlpQuery
import utils

### Bloomberg Connection

In [ ]:
con = pdblp.BCon(debug=False, port=8194, timeout=20000)
con.start();

In [ ]:
dt_fmt = '%Y%m%d'

### Bloomberg Connection

In [ ]:
bq = BlpQuery(timeout=20000).start()

In [6]:
tickers = bq.bsrch('FI:test1')

In [7]:
tickers

0     BW201748 Corp
1     BW424573 Corp
2     BW201747 Corp
3     BW201749 Corp
4     BW424575 Corp
          ...      
75    BT489121 Corp
76    BT489130 Corp
77    BT489118 Corp
78    BW487187 Corp
79    BW605083 Corp
Length: 80, dtype: object

In [8]:
flds = [
    'TICKER',
    'SECURITY_NAME',
    'GREEN_BOND_LOAN_INDICATOR',
    'ISSUE_DT',
    'ID_ISIN',
    'Z_SPRD_MID',
    'Z_SPRD_ASK',
    'Z_SPRD_BID',
    'CPN',
    'MATURITY',
    'CALC_MATURITY',
    'AMT_OUTSTANDING',
    'BB_COMPOSITE',
    'MTY_TYP',
    'CRNCY',
    'PX_LAST',
    'YLD_YTM_MID',
    'BICS_LEVEL_2_INDUSTRY_GROUP_NAME',
]

In [9]:
data = bq.bdp([*tickers], flds)

In [10]:
data

security  TICKER            SECURITY_NAME GREEN_BOND_LOAN_INDICATOR  \
0   BV224907 Corp   LXSGR     LXSGR 1 3/4 03/22/28                         N   
1   BT307159 Corp  RENAUL    RENAUL 0 1/2 07/14/25                         N   
2   BV716702 Corp   CETIN     CETIN 3 1/8 04/14/27                         N   
3   BT489138 Corp   PRXNA     PRXNA 2.778 01/19/34                         N   
4   BV630061 Corp    RIFP      RIFP 1 3/8 04/07/29                         N   
..            ...     ...                      ...                       ...   
75  BV408362 Corp      VW            VW 4 3/8 PERP                         N   
76  BW424571 Corp  COLOBD    COLOBD Float 05/19/24                         N   
77  BV408032 Corp      VW            VW 3.748 PERP                         N   
78  BV630048 Corp  CLNXSM    CLNXSM 2 1/4 04/12/26                         N   
79  BV444352 Corp  BAYNGR  BAYNGR 5 3/8 03/25/2082                         N   

     ISSUE_DT       ID_ISIN  Z_SPRD_MID  Z_SPRD_ASK  Z_SPRD_BID    CPN  \
0  2022-03-22  XS2459163619  164.168803  160.168872  168.175644  1.750   
1  2022-01-14  FR0014007KL5  165.634962  156.649217  174.647463  0.500   
2  2022-04-14  XS2468979302  240.307672  232.560333  248.083582  3.125   
3  2022-01-19  XS2430287875  362.528646  353.211483  371.932556  2.778   
4  2022-04-07  FR0014009L57   59.398482   56.408282   62.393417  1.375   
..        ...           ...         ...         ...         ...    ...   
75 2022-03-28  XS2342732646  407.010110  400.610949  413.439713  4.375   
76 2022-05-19  XS2481287394   57.381358   54.284666   60.478757  0.370   
77 2022-03-28  XS2342732562  364.807817  356.877740  372.769813  3.748   
78 2022-04-12  XS2465792294  179.267447  174.983989  183.556779  2.250   
79 2022-03-25  XS2451803063  405.272270  398.516665  412.059980  5.375   

     MATURITY CALC_MATURITY  AMT_OUTSTANDING BB_COMPOSITE    MTY_TYP CRNCY  \
0  2028-03-22    2028-03-22     6.000000e+08          BBB   CALLABLE   EUR   
1  2025-07-14    2025-07-14     7.500000e+08         BBB-   CALLABLE   EUR   
2  2027-04-14    2027-04-14     5.000000e+08          BBB   CALLABLE   EUR   
3  2034-01-19    2034-01-19     6.500000e+08         BBB-   CALLABLE   EUR   
4  2029-04-07    2029-04-07     7.500000e+08         BBB+   CALLABLE   EUR   
..        ...           ...              ...          ...        ...   ...   
75        NaT    2171-03-28     1.250000e+09           NR  PERP/CALL   EUR   
76 2024-05-19    2024-05-19     6.500000e+08           NR   CALLABLE   EUR   
77        NaT    2170-12-28     1.000000e+09         BBB-  PERP/CALL   EUR   
78 2026-04-12    2026-04-12     1.000000e+09          BB+   CALLABLE   EUR   
79 2082-03-25    2082-03-25     8.000000e+08          BB+   CALLABLE   EUR   

    PX_LAST  YLD_YTM_MID           BICS_LEVEL_2_INDUSTRY_GROUP_NAME  
0    92.759     3.124772                                  Chemicals  
1    93.000     2.852876                  Automobiles Manufacturing  
2    97.051     3.794291       Wireline Telecommunications Services  
3    77.344     5.450537                             Internet Media  
4    95.036     2.158396                            Food & Beverage  
..      ...          ...                                        ...  
75   90.747     5.306070                  Automobiles Manufacturing  
76  100.313     0.214668  Medical Equipment & Devices Manufacturing  
77   93.562     4.500090                  Automobiles Manufacturing  
78   96.983     3.082426                           Industrial Other  
79   98.040     5.798375                            Pharmaceuticals  

[80 rows x 19 columns]

In [11]:
start_date = pd.Timestamp.today() - pd.tseries.offsets.YearEnd()
end_date = pd.Timestamp.today()

In [12]:
history_flds = [
    'Z_SPRD_MID',
]

In [13]:
history_data = bq.bdh(
    securities=[*tickers], 
    fields=history_flds, 
    start_date=start_date.strftime(dt_fmt), 
    end_date=end_date.strftime(dt_fmt)
)

In [14]:
def get_first_valid(series):
    return series.loc[series.first_valid_index()]

def get_change(series):
    series = series.dropna()
    return series.iloc[1] - series.iloc[0]

In [15]:
merged_data = pd.concat([
    data.set_index('security'),
    history_data.pivot(index='date', columns='security').apply(get_first_valid).droplevel(0).rename('Z_SPRD_MID_0'),
], axis=1)

In [16]:
final_data = merged_data.assign(
    Z_SPRD_CHANGE=merged_data['Z_SPRD_MID'] - merged_data['Z_SPRD_MID_0']
)
final_data

TICKER            SECURITY_NAME GREEN_BOND_LOAN_INDICATOR  \
security                                                                   
BV224907 Corp   LXSGR     LXSGR 1 3/4 03/22/28                         N   
BT307159 Corp  RENAUL    RENAUL 0 1/2 07/14/25                         N   
BV716702 Corp   CETIN     CETIN 3 1/8 04/14/27                         N   
BT489138 Corp   PRXNA     PRXNA 2.778 01/19/34                         N   
BV630061 Corp    RIFP      RIFP 1 3/8 04/07/29                         N   
...               ...                      ...                       ...   
BV408362 Corp      VW            VW 4 3/8 PERP                         N   
BW424571 Corp  COLOBD    COLOBD Float 05/19/24                         N   
BV408032 Corp      VW            VW 3.748 PERP                         N   
BV630048 Corp  CLNXSM    CLNXSM 2 1/4 04/12/26                         N   
BV444352 Corp  BAYNGR  BAYNGR 5 3/8 03/25/2082                         N   

                ISSUE_DT       ID_ISIN  Z_SPRD_MID  Z_SPRD_ASK  Z_SPRD_BID  \
security                                                                     
BV224907 Corp 2022-03-22  XS2459163619  164.168803  160.168872  168.175644   
BT307159 Corp 2022-01-14  FR0014007KL5  165.634962  156.649217  174.647463   
BV716702 Corp 2022-04-14  XS2468979302  240.307672  232.560333  248.083582   
BT489138 Corp 2022-01-19  XS2430287875  362.528646  353.211483  371.932556   
BV630061 Corp 2022-04-07  FR0014009L57   59.398482   56.408282   62.393417   
...                  ...           ...         ...         ...         ...   
BV408362 Corp 2022-03-28  XS2342732646  407.010110  400.610949  413.439713   
BW424571 Corp 2022-05-19  XS2481287394   57.381358   54.284666   60.478757   
BV408032 Corp 2022-03-28  XS2342732562  364.807817  356.877740  372.769813   
BV630048 Corp 2022-04-12  XS2465792294  179.267447  174.983989  183.556779   
BV444352 Corp 2022-03-25  XS2451803063  405.272270  398.516665  412.059980   

                 CPN   MATURITY CALC_MATURITY  AMT_OUTSTANDING BB_COMPOSITE  \
security                                                                      
BV224907 Corp  1.750 2028-03-22    2028-03-22     6.000000e+08          BBB   
BT307159 Corp  0.500 2025-07-14    2025-07-14     7.500000e+08         BBB-   
BV716702 Corp  3.125 2027-04-14    2027-04-14     5.000000e+08          BBB   
BT489138 Corp  2.778 2034-01-19    2034-01-19     6.500000e+08         BBB-   
BV630061 Corp  1.375 2029-04-07    2029-04-07     7.500000e+08         BBB+   
...              ...        ...           ...              ...          ...   
BV408362 Corp  4.375        NaT    2171-03-28     1.250000e+09           NR   
BW424571 Corp  0.370 2024-05-19    2024-05-19     6.500000e+08           NR   
BV408032 Corp  3.748        NaT    2170-12-28     1.000000e+09         BBB-   
BV630048 Corp  2.250 2026-04-12    2026-04-12     1.000000e+09          BB+   
BV444352 Corp  5.375 2082-03-25    2082-03-25     8.000000e+08          BB+   

                 MTY_TYP CRNCY  PX_LAST  YLD_YTM_MID  \
security                                               
BV224907 Corp   CALLABLE   EUR   92.759     3.124772   
BT307159 Corp   CALLABLE   EUR   93.000     2.852876   
BV716702 Corp   CALLABLE   EUR   97.051     3.794291   
BT489138 Corp   CALLABLE   EUR   77.344     5.450537   
BV630061 Corp   CALLABLE   EUR   95.036     2.158396   
...                  ...   ...      ...          ...   
BV408362 Corp  PERP/CALL   EUR   90.747     5.306070   
BW424571 Corp   CALLABLE   EUR  100.313     0.214668   
BV408032 Corp  PERP/CALL   EUR   93.562     4.500090   
BV630048 Corp   CALLABLE   EUR   96.983     3.082426   
BV444352 Corp   CALLABLE   EUR   98.040     5.798375   

                        BICS_LEVEL_2_INDUSTRY_GROUP_NAME  Z_SPRD_MID_0  \
security                                                                 
BV224907 Corp                                  Chemicals        99.224   
BT307159 Corp                  A

In [17]:
final_data.to_clipboard()

In [18]:
final_data.to_excel('daily.xlsx')